In [1]:
# general functions:
def downsample(time_series,res = '0.2S'):
    # downsamples time_series pandas data frame 
    # time_series: a pandas data.frame for which there is a column 'timestamps' with numeric timestamps
    # res: desired resolution in time after downsampling
    Nvalues = len(time_series.index)
    samplerate = 1/ ((time_series.timestamp[Nvalues-1] - time_series.timestamp[0]) / Nvalues)
    timestart = dt.datetime(1970, 1, 1, 0, 0, 0, 0) #dt.datetime.now()
    start = pd.Timestamp(timestart)
    end = pd.Timestamp(timestart + dt.timedelta(seconds=Nvalues/samplerate))
    t = np.linspace(start.value, end.value, Nvalues)
    t = pd.to_datetime(t)
    time_series['time'] = t
    time_series = time_series.resample(res,on='time').mean() # downsample to 0.2 second intervals
    time_series.index.name = 'time'
    time_series.reset_index(inplace=True)
    return time_series

# Merge features from various modalities into single database

## 1. Load feature data from Video

Load OpenFace output data and load features in pandas dataframe: So far copied from: https://github.com/emrecdem/exploface/blob/master/TUTORIALS/tutorial1.ipynb

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import exploface
import datetime as dt
import numpy as np

In [3]:
exploface.__version__

'0.0.0.dev6'

In [4]:
# specify some paths
emrecdemStudyDataFolder = "/media/sf_sharedfolder/Emotion/emrecdemstudydata"
openface_outputfolder = emrecdemStudyDataFolder + "/OpenFaceOutput"

In [5]:
# Search for files with csv extension, because we are only interested in those
# This assumes that there are no other csv files in folder other than the ones produced by OpenFace.
from os import listdir

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

filenames = find_csv_filenames(openface_outputfolder)

In [6]:
filenames

['P18_S2_IAPS_HAPPY_Cfront.csv',
 'P18_S2_IAPS_SAD_Cfront.csv',
 'P21_S2_IAPS_HAPPY_C1.csv',
 'P21_S2_IAPS_SAD_C1.csv']

In [177]:
# Potential loop over filenames, for now just one index
fileindex = 3
deleteDataBaseEntries = False

In [178]:
# select one file to process (in the future this can be a loop over all the files)
openface_file = openface_outputfolder + '/' + filenames[fileindex]
openface_features = exploface.get_feature_time_series(openface_file)

In [179]:
# extract participant id (PID) and experiment id (EXP) from filename:
PID = filenames[fileindex].split("_")[0].split("P")[1]
EXP = ''.join(filenames[fileindex].split("_")[1:4]) # extract integers from filename

In [180]:
EXP = EXP.split(".csv")[0] # remove .csv at the end
print(EXP)

S2IAPSSAD


In [181]:
PID

'21'

In [182]:
openface_features = downsample(openface_features,res = '0.2S')

In [183]:
# tidy up data frame:
filter_col = [col for col in openface_features if col.startswith('AU')]
filter_col.insert(0,'time')
filter_col.insert(0,'participant_id')
filter_col.insert(0,'experiment_id')
openface_features['participant_id'] = PID
openface_features['experiment_id'] = EXP
openface_features = openface_features[filter_col]
openface_features.columns = openface_features.columns.str.replace('_', '')
openface_features = openface_features.rename(columns = {'experimentid':'experiment_id'})
openface_features = openface_features.rename(columns = {'participantid':'participant_id'})

In [184]:
openface_features.head()

,experiment_id,participant_id,time,AU01r,AU02r,AU04r,AU05r,AU06r,AU07r,AU09r,...,AU12c,AU14c,AU15c,AU17c,AU20c,AU23c,AU25c,AU26c,AU28c,AU45c
0,S2IAPSSAD,21,1970-01-01 00:00:00.000,0.0,0.0,1.698,0.0,1.156,1.526,0.310,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,S2IAPSSAD,21,1970-01-01 00:00:00.200,0.0,0.0,1.820,0.0,1.212,1.714,0.218,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,S2IAPSSAD,21,1970-01-01 00:00:00.400,0.0,0.0,1.668,0.0,1.340,1.980,0.218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.6
3,S2IAPSSAD,21,1970-01-01 00:00:00.600,0.0,0.0,1.748,0.0,1.288,2.158,0.192,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
4,S2IAPSSAD,21,1970-01-01 00:00:00.800,0.0,0.0,1.840,0.0,1.140,1.860,0.102,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. Load feature data from audio

Process data with Librosa in pandas dataframe: So far copied from: https://github.com/emrecdem/explibrosa/blob/master/TUTORIALS/tutorial1.ipynb

In [185]:
import matplotlib.pyplot as plt
import os
import explibrosa

In [186]:
explibrosa.__version__

'0.0.0.dev1'

Find wav file that matches the csv file produced by OpenFace based on the assumption that filenames are identical except from file extension

In [187]:
transformfilename = filenames[fileindex]

In [188]:
audiofile_name = transformfilename.replace('.csv','.wav').replace('_Cfront','').replace('_Cside','').replace('_C1','')

In [189]:
audiofile_name

'P21_S2_IAPS_SAD.wav'

In [190]:
import subprocess
 
# Set up find command
findCMD = 'find ' + emrecdemStudyDataFolder + ' -name ' + audiofile_name 
out = subprocess.Popen(findCMD,shell=True,stdin=subprocess.PIPE, 
                        stdout=subprocess.PIPE,stderr=subprocess.PIPE)
# Get standard out and error
(stdout, stderr) = out.communicate()
# Save found files to list
filelist = stdout.decode().split()

In [191]:
findCMD

'find /media/sf_sharedfolder/Emotion/emrecdemstudydata -name P21_S2_IAPS_SAD.wav'

In [192]:
audiofiles_fullPaths = filelist # probably the list has only one filename
audio_file = audiofiles_fullPaths[0]

In [193]:
audiofiles_fullPaths

['/media/sf_sharedfolder/Emotion/emrecdemstudydata/P21/S2/Audio/P21_S2_IAPS_SAD.wav']

In [194]:
os.path.isfile(audio_file)

True

In [195]:
explibrosa.get_info(audio_file)

{'#frames': 2632496, 'duration (min)': 2.7, 'Sample freq (kHz)': 16.0}

In [196]:
time_series = explibrosa.get_feature_time_series(audio_file)

Running librosa (no results found on disk)
RMS energy
     1.48 seconds
Zero crossing
     1.66 seconds
Pitches
     8.12 seconds
  Pitches smoothing
     8.45 seconds
TOTAL execution time: 0.14 min


In [197]:
time_series = downsample(time_series,res = '0.2S')

In [198]:
# Add columns and tidy up column names
time_series['participant_id'] = PID
time_series['experiment_id'] = EXP
time_series = time_series.rename(columns={'zrc': 'zcrate'})
time_series.drop(['timestamp'], axis = 1, inplace = True, errors = 'ignore')
time_series = time_series[['participant_id','experiment_id','time','pitch','rmse','zcrate']]

In [199]:
time_series.head()

,participant_id,experiment_id,time,pitch,rmse,zcrate
0,21,S2IAPSSAD,1970-01-01 00:00:00.000,372.844088,0.008293,0.276027
1,21,S2IAPSSAD,1970-01-01 00:00:00.200,347.708417,0.007039,0.258447
2,21,S2IAPSSAD,1970-01-01 00:00:00.400,230.505765,0.007674,0.251370
3,21,S2IAPSSAD,1970-01-01 00:00:00.600,374.712016,0.013936,0.100228
4,21,S2IAPSSAD,1970-01-01 00:00:00.800,561.551068,0.010230,0.142922


## 3. Store in database


http://www.sqlitetutorial.net/sqlite-python/create-tables/

In [200]:
import sqlite3

In [201]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

In [202]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [203]:
def main():
    # Note time in text datatype, which will facilitate ISO8601 "YYYY-MM-DD HH:MM:SS.SSS"
    database = "/media/sf_sharedfolder/Emotion/emrecdemstudydata/emrecdemdb.db"
     
    sql_create_participants_table = """ CREATE TABLE IF NOT EXISTS participants (
                                        id integer PRIMARY KEY,
                                        age real
                                    ); """
    
    sql_create_experiments_table = """ CREATE TABLE IF NOT EXISTS experiments (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL
                                    ); """
    sql_create_videofeatures_table = """ CREATE TABLE IF NOT EXISTS videofeatures (
                                        id integer PRIMARY KEY,
                                        participant_id integer NOT NULL,
                                        experiment_id text NOT NULL,
                                        time text,
                                        AU01r real, AU02r real, AU04r real,
                                        AU05r real, AU06r real, AU07r real,
                                        AU09r real, AU10r real, AU12r real,
                                        AU14r real, AU15r real, AU17r real,
                                        AU20r real, AU23r real, AU25r real,
                                        AU26r real, AU45r real, AU01c real,
                                        AU02c real, AU04c real, AU05c real,
                                        AU06c real, AU07c real, AU09c real,
                                        AU10c real, AU12c real, AU14c real,
                                        AU15c real, AU17c real, AU20c real,
                                        AU23c real, AU25c real, AU26c real,
                                        AU28c real, AU45c real,                                        
                                        FOREIGN KEY (participant_id) REFERENCES participants (id),
                                        FOREIGN KEY (experiment_id) REFERENCES experiments (id)
                                    ); """
    
    sql_create_audiofeatures_table = """ CREATE TABLE IF NOT EXISTS audiofeatures (
                                        id integer PRIMARY KEY,
                                        participant_id integer NOT NULL,
                                        experiment_id text NOT NULL,
                                        time text, 
                                        pitch real,
                                        rmse real,
                                        zcrate real,
                                        FOREIGN KEY (participant_id) REFERENCES participants (id),
                                        FOREIGN KEY (experiment_id) REFERENCES experiments (id)
                                    ); """
        
    sql_create_facslabels_table = """ CREATE TABLE IF NOT EXISTS facslabels (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL
                                    ); """
 
    sql_create_facsencoding_table = """CREATE TABLE IF NOT EXISTS tasks (
                                    id integer PRIMARY KEY,
                                    participant_id integer NOT NULL,
                                    experiment_id integer NOT NULL,
                                    timestamp text,
                                    emotion_id integer NOT NULL,
                                    FOREIGN KEY (participant_id) REFERENCES participants (id),
                                    FOREIGN KEY (experiment_id) REFERENCES experiments (id),
                                    FOREIGN KEY (emotion_id) REFERENCES facslabels (id)
                                );"""
 
    # create a database connection
    conn = create_connection(database)
    if conn is not None:
        # create participants table
        create_table(conn, sql_create_participants_table)
        # create experiments table
        create_table(conn, sql_create_experiments_table)
        create_table(conn, sql_create_videofeatures_table)
        create_table(conn, sql_create_audiofeatures_table)
        create_table(conn, sql_create_facslabels_table)
        create_table(conn, sql_create_facsencoding_table)
    else:
        print("Error! cannot create the database connection.")

In [204]:

if __name__ == '__main__':
    main()

In [205]:
# Now on command line: sqlite3 pathtomydatabase.db
# should give access to database
# and .tables should show overview of the tables as created

In [206]:
#https://www.dataquest.io/blog/python-pandas-databases/
database = "/media/sf_sharedfolder/Emotion/emrecdemstudydata/emrecdemdb.db"

In [207]:
conn = create_connection(database) # create connection
cur = conn.cursor() # create cursor

### Add Audio features to database

In [208]:
# delete contents of table for testing purposes, later we obviously do not want to do this
if (deleteDataBaseEntries == True):
    deleterows = cur.execute("DELETE FROM audiofeatures;").fetchall()

In [209]:
results = cur.execute("select * from audiofeatures limit 5;").fetchall()
rowcount = cur.execute("select count(*) from audiofeatures;").fetchall()
colnames = cur.execute("PRAGMA table_info(audiofeatures);").fetchall()

In [210]:
print("Top 5 rows of audiofeatures table:")
print(results)
print("\nRow count:")
print(rowcount[0][0])

Top 5 rows of audiofeatures table:
[(1, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00', 271.2116057159737, 0.008785701356828213, 0.1180952380952381), (2, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00.200000', 260.67761899075373, 0.008010043762624264, 0.1419047619047619), (3, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00.400000', 239.59336760187193, 0.0033461761195212603, 0.18642857142857144), (4, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00.600000', 205.65226665554346, 0.010653898119926453, 0.17619047619047618), (5, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00.800000', 349.66176893618893, 0.02047872543334961, 0.15357142857142855)]

Row count:
4507


In [211]:
print("\nColumn names:")
print(colnames)


Column names:
[(0, 'id', 'integer', 0, None, 1), (1, 'participant_id', 'integer', 1, None, 0), (2, 'experiment_id', 'text', 1, None, 0), (3, 'time', 'text', 0, None, 0), (4, 'pitch', 'real', 0, None, 0), (5, 'rmse', 'real', 0, None, 0), (6, 'zcrate', 'real', 0, None, 0)]


In [212]:
# move time_series pandas data frame to the database
time_series.to_sql("audiofeatures", conn, index= False, if_exists="append") #"replace"

In [213]:
results = cur.execute("select * from audiofeatures limit 5;").fetchall()
rowcount = cur.execute("select count(*) from audiofeatures;").fetchall()
colnames = cur.execute("PRAGMA table_info(audiofeatures);").fetchall()

In [214]:
print("Top 5 rows of audiofeatures table:")
print(results)
print("\nRow count:")
print(rowcount[0][0])
print("\nDimensions of pandas dataframe:")
print(time_series.shape)
print("\nColumn names:")
print(colnames)

Top 5 rows of audiofeatures table:
[(1, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00', 271.2116057159737, 0.008785701356828213, 0.1180952380952381), (2, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00.200000', 260.67761899075373, 0.008010043762624264, 0.1419047619047619), (3, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00.400000', 239.59336760187193, 0.0033461761195212603, 0.18642857142857144), (4, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00.600000', 205.65226665554346, 0.010653898119926453, 0.17619047619047618), (5, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00.800000', 349.66176893618893, 0.02047872543334961, 0.15357142857142855)]

Row count:
5330

Dimensions of pandas dataframe:
(823, 6)

Column names:
[(0, 'id', 'integer', 0, None, 1), (1, 'participant_id', 'integer', 1, None, 0), (2, 'experiment_id', 'text', 1, None, 0), (3, 'time', 'text', 0, None, 0), (4, 'pitch', 'real', 0, None, 0), (5, 'rmse', 'real', 0, None, 0), (6, 'zcrate', 'real', 0, None, 0)]


### Add video data to database

In [215]:
# delete contents of table for testing purposes, later we obviously do not want to do this
if (deleteDataBaseEntries == True):
    deleterows = cur.execute("DELETE FROM videofeatures;").fetchall()

In [216]:
results = cur.execute("select * from videofeatures limit 5;").fetchall()
rowcount = cur.execute("select count(*) from videofeatures;").fetchall()
colnames = cur.execute("PRAGMA table_info(videofeatures);").fetchall()

In [217]:
print("Top 5 rows of videofeatures table:")
print(results)
print("\nRow count:")
print(rowcount[0][0])
print("\nColumn names:")
print(colnames)

Top 5 rows of videofeatures table:
[(1, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00', 0.0, 0.0, 0.986, 0.0, 0.606, 0.016, 0.02, 0.916, 0.6340000000000001, 1.1380000000000001, 0.44400000000000006, 1.054, 0.516, 0.8, 0.9039999999999999, 2.7339999999999995, 0.22599999999999998, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.6, 0.6, 0.0, 0.6, 1.0, 1.0, 1.0, 0.8, 1.0, 0.4, 0.0, 0.0), (2, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00.200000', 0.0, 0.0, 1.07, 0.0, 0.568, 0.11400000000000002, 0.028000000000000004, 1.346, 0.642, 1.5260000000000002, 0.37399999999999994, 1.206, 0.442, 0.8480000000000001, 0.30199999999999994, 2.8240000000000003, 0.21799999999999997, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.6, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6, 0.0, 0.0, 0.0), (3, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00.400000', 0.0, 0.0, 1.278, 0.0, 0.246, 0.0, 0.0, 1.214, 0.45, 1.5699999999999998, 0.384, 1.658, 0.41600000000000004, 1.766, 0.008, 2.02, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0

In [218]:
openface_features.head()

,experiment_id,participant_id,time,AU01r,AU02r,AU04r,AU05r,AU06r,AU07r,AU09r,...,AU12c,AU14c,AU15c,AU17c,AU20c,AU23c,AU25c,AU26c,AU28c,AU45c
0,S2IAPSSAD,21,1970-01-01 00:00:00.000,0.0,0.0,1.698,0.0,1.156,1.526,0.310,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,S2IAPSSAD,21,1970-01-01 00:00:00.200,0.0,0.0,1.820,0.0,1.212,1.714,0.218,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,S2IAPSSAD,21,1970-01-01 00:00:00.400,0.0,0.0,1.668,0.0,1.340,1.980,0.218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.6
3,S2IAPSSAD,21,1970-01-01 00:00:00.600,0.0,0.0,1.748,0.0,1.288,2.158,0.192,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
4,S2IAPSSAD,21,1970-01-01 00:00:00.800,0.0,0.0,1.840,0.0,1.140,1.860,0.102,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [219]:
# move time_series pandas data frame to the database
openface_features.to_sql("videofeatures", conn, index=False,if_exists="append") #"replace"

In [220]:
results = cur.execute("select * from videofeatures limit 5;").fetchall()
rowcount = cur.execute("select count(*) from videofeatures;").fetchall()

In [221]:
print("Top 5 rows of videofeatures table:")
print(results)
print("\nRow count:")
print(rowcount[0][0])
print("\nDimensions of pandas dataframe:")
print(time_series.shape)

Top 5 rows of videofeatures table:
[(1, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00', 0.0, 0.0, 0.986, 0.0, 0.606, 0.016, 0.02, 0.916, 0.6340000000000001, 1.1380000000000001, 0.44400000000000006, 1.054, 0.516, 0.8, 0.9039999999999999, 2.7339999999999995, 0.22599999999999998, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.6, 0.6, 0.0, 0.6, 1.0, 1.0, 1.0, 0.8, 1.0, 0.4, 0.0, 0.0), (2, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00.200000', 0.0, 0.0, 1.07, 0.0, 0.568, 0.11400000000000002, 0.028000000000000004, 1.346, 0.642, 1.5260000000000002, 0.37399999999999994, 1.206, 0.442, 0.8480000000000001, 0.30199999999999994, 2.8240000000000003, 0.21799999999999997, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.6, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6, 0.0, 0.0, 0.0), (3, 18, 'S2IAPSHAPPY', '1970-01-01 00:00:00.400000', 0.0, 0.0, 1.278, 0.0, 0.246, 0.0, 0.0, 1.214, 0.45, 1.5699999999999998, 0.384, 1.658, 0.41600000000000004, 1.766, 0.008, 2.02, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0

In [222]:
cur.close()
conn.close()